In [65]:
import sagemaker
import boto3

In [5]:
from sagemaker.session import s3_input,Session

In [48]:
from sagemaker.amazon.amazon_estimator import image_uris

In [21]:
bucket_name= "bankml"
region = boto3.session.Session().region_name

In [22]:
region

'us-east-1'

#### Making a bucket 

In [23]:
s3 = boto3.resource("s3")
try:
    if region == 'us-east-1':
        s3.create_bucket(Bucket = bucket_name )
        print("Succesfullly bucket created ")
        
except Exception as e: 
    print("s3 error : ", e)

Succesfullly bucket created 


### Setting output path for the model 

In [24]:
prefix= 'xg-boost-as-a-built-in-algo' 

In [66]:
output_path = "s3://{}/{}/output".format(bucket_name, prefix)
print(output_path)

s3://bankml/xg-boost-as-a-built-in-algo/output


### Downloading and storing data 

In [26]:
import pandas as pd
import urllib

In [28]:
# Downloading from url
try:
    urllib.request.urlretrieve("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print("Downloaded Succesfully ")
except Exception as e:
    print(" Error Occured : ", e)

In [29]:
# storing data in pandas
try:
    model_data = pd.read_csv('./bank_clean.csv', index_col=0)
    print("Read succssfully")
except Exception as e :
    print("Error : ", e)

Read succssfully


#### Split the data into train and test 

In [30]:
import numpy as np  

In [31]:
train_data , test_data = np.split(model_data.sample(frac= 1, random_state= 1729), [int(0.7* len(model_data))])

####  Removing the yes and no outputs 

In [36]:
pd.concat([train_data['y_yes'], train_data.drop(['y_yes', 'y_no'] , axis=1)], axis= 1 ).to_csv('train.csv', index = False , header = False)

### uploading train.csv to bucket 

In [39]:
import os
boto3.Session().resource("s3").Bucket(bucket_name).Object(os.path.join(prefix,'train/train.csv')).upload_file('train.csv')

### Create path to the file

In [55]:
s3_input_train= sagemaker.TrainingInput(s3_data ='s3://{}/{}/train'.format(bucket_name, prefix), content_type = 'csv')

same for test

In [42]:
import os
pd.concat([test_data['y_yes'], test_data.drop(['y_yes', 'y_no'] , axis=1)], axis= 1 ).to_csv('test.csv', index = False , header = False)
boto3.Session().resource("s3").Bucket(bucket_name).Object(os.path.join(prefix,'test/test.csv')).upload_file('test.csv')

In [56]:
s3_input_test= sagemaker.TrainingInput(s3_data ='s3://{}/{}/test'.format(bucket_name, prefix), content_type = 'csv')

### Creating XG Boost , it is an inbuilt alg  

pulling the container

In [57]:
container = image_uris.retrieve(region=boto3.Session().region_name, framework='xgboost', version='1.0-1')

### Hyper parameters

In [62]:
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

### Construuct the estimator

In [67]:
estimator = sagemaker.estimator.Estimator(image_uri= container,
                                         hyperparameters= hyperparameters,
                                         role = sagemaker.get_execution_role(),
                                         instance_count= 1,
                                         instance_type = 'ml.m5.2xlarge',
                                         volume_size= 5,
                                         output_path= output_path,
                                         use_spot_instances= True,
                                         max_run= 300,
                                         max_wait= 600)

In [68]:
estimator.fit({'train':s3_input_train , 'validation':s3_input_test})

2022-04-29 20:29:58 Starting - Starting the training job...
2022-04-29 20:30:10 Starting - Launching requested ML instancesProfilerReport-1651264198: InProgress
.........
2022-04-29 20:31:40 Starting - Preparing the instances for training......
2022-04-29 20:32:58 Downloading - Downloading input data
2022-04-29 20:32:58 Training - Downloading the training image......
2022-04-29 20:33:56 Uploading - Uploading generated training model
2022-04-29 20:33:56 Completed - Training job completed
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV in

### Deploy the model

In [69]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')


----------!

### Predicting from test data

In [74]:
from sagemaker.serializers import CSVSerializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.serializer = CSVSerializer(content_type='text/csv')# set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


### Confution Matrix

In [75]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))



Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 

